In [8]:
import cassandra
from cassandra import ConsistencyLevel
from cassandra.cluster import Cluster
from cassandra.query import SimpleStatement, BatchStatement

from joblib import Parallel, delayed                                                                                                                                
import os
import sys
import json
import utils
import pickle

from db import *
from glob import glob
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

In [2]:
"""
sudo docker run --name cassandra -p 127.0.0.1:9042:9042 -e CASSANDRA_CLUSTER_NAME=GangaTest -e CASSANDRA_ENDPOINT_SNITCH=GossipingPropertyFileSnitch -e CASSANDRA_DC=datacenter1 -d cassandra
"""
def cassandra_connection():
    """
    Connection object for Cassandra
    :return: session, cluster
    """
    cluster = Cluster(['127.0.0.1'], port=9042)
    session = cluster.connect()
    session.execute("""
        CREATE KEYSPACE IF NOT EXISTS fakehealthcareorg
        WITH REPLICATION =
        { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }
        """)
    session.set_keyspace('fakehealthcareorg')
    return session, cluster


In [3]:
session, cluster = cassandra_connection()

In [4]:
session.execute("""create keyspace IF NOT EXISTS dev
... with replication = {'class':'SimpleStrategy','replication_factor':1};""")

In [5]:
session.execute("""use dev;""")

In [6]:
session.execute("""DROP TABLE IF EXISTS JOB""")
session.execute("""DROP TABLE IF EXISTS BLOB""")

In [18]:
SIZE = 10
DB = "postgres"
jobs, blobs = utils.load_pickle_data(size=SIZE)

The runtime for load_pickle_data was 0.06233572959899902 seconds.


In [21]:
jobs = pickle.load(open("../../data/rows.pkl", "rb"))[:100]
jobs[:10]

[['0',
  "'completed'",
  'None',
  1,
  "'85c74d55153c2b535aa07b2067c9381e'",
  "'Done'",
  "'LCG.CNAF.it'",
  "''"],
 ['0',
  "'completed'",
  'None',
  1,
  "'85c74d55153c2b535aa07b2067c9381e'",
  "'Done'",
  "'LCG.CNAF.it'",
  "''"],
 ['0',
  "'completed'",
  'None',
  1,
  "'85c74d55153c2b535aa07b2067c9381e'",
  "'Done'",
  "'LCG.CNAF.it'",
  "''"],
 ['0',
  "'completed'",
  'None',
  1,
  "'85c74d55153c2b535aa07b2067c9381e'",
  "'Done'",
  "'LCG.CNAF.it'",
  "''"],
 ['0',
  "'completed'",
  'None',
  1,
  "'85c74d55153c2b535aa07b2067c9381e'",
  "'Done'",
  "'LCG.CNAF.it'",
  "''"],
 ['0',
  "'completed'",
  'None',
  1,
  "'85c74d55153c2b535aa07b2067c9381e'",
  "'Done'",
  "'LCG.CNAF.it'",
  "''"],
 ['0',
  "'completed'",
  'None',
  1,
  "'85c74d55153c2b535aa07b2067c9381e'",
  "'Done'",
  "'LCG.CNAF.it'",
  "''"],
 ['0',
  "'completed'",
  'None',
  1,
  "'85c74d55153c2b535aa07b2067c9381e'",
  "'Done'",
  "'LCG.CNAF.it'",
  "''"],
 ['0',
  "'completed'",
  'None',
  1,
  "'85c74

In [16]:
session, cluster = cassandra_connection()
session = create_tables(session)

The runtime for cassandra_connection was 0.038602590560913086 seconds.
The runtime for create_tables was 0.0076138973236083984 seconds.


In [17]:
jobs[:10]

[]

In [15]:
query = session.prepare("""
    INSERT INTO JOB(jid, status, name, subjobs, application, backend, backend_actualCE, comment)
    VALUES          (?,      ?,    ?,       ?,           ?,       ?,                ?,       ?)
""")

if jobs:
    with tqdm(total=len(jobs)) as progress:
        for i, row in enumerate(jobs):
            row[0] = i+1
            session.execute(query.bind(row))
            progress.update(1)
